--- Day 5: If You Give A Seed A Fertilizer ---

You take the boat and find the gardener right where you were told he would be: managing a giant "garden" that looks more to you like a farm.

"A water source? Island Island is the water source!" You point out that Snow Island isn't receiving any water.

"Oh, we had to stop the water because we ran out of sand to filter it with! Can't make snow with dirty water. Don't worry, I'm sure we'll get more sand soon; we only turned off the water a few days... weeks... oh no." His face sinks into a look of horrified realization.

"I've been so busy making sure everyone here has food that I completely forgot to check why we stopped getting more sand! There's a ferry leaving soon that is headed over in that direction - it's much faster than your boat. Could you please go check it out?"

You barely have time to agree to this request when he brings up another. "While you wait for the ferry, maybe you can help us with our food production problem. The latest Island Island Almanac just arrived and we're having trouble making sense of it."

The almanac (your puzzle input) lists all of the seeds that need to be planted. It also lists what type of soil to use with each kind of seed, what type of fertilizer to use with each kind of soil, what type of water to use with each kind of fertilizer, and so on. Every type of seed, soil, fertilizer and so on is identified with a number, but numbers are reused by each category - that is, soil 123 and fertilizer 123 aren't necessarily related to each other.

In [180]:
day5testFile = open('day5.txt', 'r').read()


In [188]:
inputs = [int(x) for x in day5testFile.split('\n\n')[0].split(':')[1].strip().split(' ')]

#inputs part 2 are interval
inputsPart2 = []
for i in range (0,len(inputs), 2):
    inputsPart2.append((inputs[i], inputs[i] + inputs[i+1]-1))

In [182]:
maps = [x.split(':')[1].strip().split('\n') for x in day5testFile.split('\n\n')[1:]]


In [193]:
def getIntersectionInterval(intervalA: tuple[int, int], intervalB: tuple[int, int]) -> tuple[int,int] | None:
    if int(intervalA[0]) >= int(intervalB[0]) and int(intervalA[0]) <= (intervalB[1]):
        return (max(intervalA[0], intervalB[0]), min(intervalA[1], intervalB[1]))
    if intervalA[1] >= intervalB[0] and intervalA[1]<= intervalB[1]:
        return (max(intervalA[0], intervalB[0]), min(intervalA[1], intervalB[1]))
    return None

In [195]:
def removeIntervalFromInterval(original: tuple[int, int], toRemove:[int,int]) -> list[tuple[int, int]]:
    if original[0] >= toRemove[0] and original[1] <= toRemove[1]:
        return []
    if original[0] < toRemove[0] and original[1] > toRemove[1]:
        return [(original[0], toRemove[0]-1), (toRemove[1]+1,original[1])]
    if original[0] < toRemove[0]:
        return [(original[0],toRemove[0]-1)]
    return [(toRemove[1]+1,original[1])]


In [190]:
def applyMap(inp, maps,index):
    currentMap = maps[index];
    nextInputs =[]
    toProcess = inp
    while toProcess != []:
        currentlyProcessing = toProcess.pop()
        for func in currentMap:
            dest, source, rng = [int(x) for x in func.split(' ')]
            functionDestInterval = (dest, dest + rng -1)
            functionSourceInterval = (source, source + rng -1)
            
            #find intersection between current input and source interval
            intersection = getIntersectionInterval(currentlyProcessing, functionSourceInterval)
            if intersection != None: 
                toAdd = dest - source
                nextInputs.append((intersection[0] + toAdd , intersection[1] + toAdd))
                #for numbers outside matched interval continue checking them against other maps
                for addToProcess in removeIntervalFromInterval(currentlyProcessing, intersection):
                    toProcess.append(addToProcess)
                currentlyProcessing = None
                break
                
    #pass unprocessed inputs to next map
    if currentlyProcessing != None:
        nextInputs += [currentlyProcessing]
        
    #return minimal location for input
    if index +1 >= len(maps):
        return min(nextInputs)
    
    #apply recusively 
    return applyMap(nextInputs,maps, index+1)
        

In [134]:
res = []
for inp in inputs:
    for m in maps:
        for func in m:
            dest, source, rng = func.split(' ')
            if int(inp) >= int(source) and int(inp) <= (int(source) + int(rng)-1):
                inp = int(dest) - int(source) + int(inp)
                break
    res.append(inp)
print(min(res))  

346433842


--- Part Two ---

Everyone will starve if you only plant such a small number of seeds. Re-reading the almanac, it looks like the seeds: line actually describes ranges of seed numbers.

The values on the initial seeds: line come in pairs. Within each pair, the first value is the start of the range and the second value is the length of the range. So, in the first line of the example above:

seeds: 79 14 55 13
This line describes two ranges of seed numbers to be planted in the garden. The first range starts with seed number 79 and contains 14 values: 79, 80, ..., 91, 92. The second range starts with seed number 55 and contains 13 values: 55, 56, ..., 66, 67.

Now, rather than considering four seed numbers, you need to consider a total of 27 seed numbers.

In the above example, the lowest location number can be obtained from seed number 82, which corresponds to soil 84, fertilizer 84, water 84, light 77, temperature 45, humidity 46, and location 46. So, the lowest location number is 46.

Consider all of the initial seed numbers listed in the ranges on the first line of the almanac. What is the lowest location number that corresponds to any of the initial seed numbers?



In [197]:
minimal = 9999999999999
for inp in inputsPart2:
    res = applyMap([inp], maps, 0)
    minimal = min(minimal, res[0])
print(minimal)
        

60294664
